XGBoost 

LightGBM

In [6]:
import pandas as pd
import numpy as np

In [42]:
data=pd.read_csv(r"C:\Users\Public\project\data\data.csv")
data.head(2)

,index,Date,Close,High,Low,Open,Volume
0,0,2015-01-01,1.209863,1.209863,1.209863,1.209863,0
1,1,2015-01-02,1.208941,1.208956,1.201080,1.208868,0


In [43]:
data.columns

Index(['index', 'Date', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

In [44]:

data['return']=data['Close'].pct_change()
data['log_return'] = np.log(data['Close']).diff()
data['target']=data['log_return'].shift(-1)


data["sma5"] = data["Close"].rolling(5).mean()
data["sma10"] = data["Close"].rolling(30).mean()
data['lag1'] = data['log_return'].shift(1)
data['lag2'] = data['log_return'].shift(2)
data['vol5'] = data['log_return'].rolling(5).std()
data['vol10'] = data['log_return'].rolling(30).std()




data = data.dropna().reset_index(drop=True)


X = data[["sma5", "sma10",'lag1','lag2','vol5','vol10']]
y = data['target']

train_size=int(0.7*len(X))
X_train=X[:train_size]
X_test=X[train_size:]
y_train=y[:train_size]
y_test=y[train_size:]





data.head()



,index,Date,Close,High,Low,Open,Volume,return,log_return,target,sma5,sma10,lag1,lag2,vol5,vol10
0,30,2015-02-12,1.131196,1.139819,1.130621,1.130825,0,-0.000882,-0.000883,0.008098,1.134945,1.155303,-0.000442,0.001109,0.009672,0.008831
1,31,2015-02-13,1.140394,1.145001,1.138200,1.140485,0,0.008131,0.008098,-0.000342,1.133584,1.153018,-0.000883,-0.000442,0.007937,0.009028
2,32,2015-02-16,1.140004,1.143500,1.139705,1.139705,0,-0.000342,-0.000342,-0.005809,1.135297,1.151196,0.008098,-0.000883,0.003759,0.008833
3,33,2015-02-17,1.133401,1.144230,1.132516,1.133363,0,-0.005792,-0.005809,0.006219,1.135438,1.149180,-0.000342,0.008098,0.005008,0.008865
4,34,2015-02-18,1.140472,1.141500,1.134780,1.140498,0,0.006238,0.006219,-0.000410,1.137093,1.147611,-0.005809,-0.000342,0.005663,0.008953


In [45]:
X_train.shape

(1802, 6)

In [46]:
X.corr()

,sma5,sma10,lag1,lag2,vol5,vol10
sma5,1.000000,0.969239,-0.006062,0.014415,-0.220904,-0.344699
sma10,0.969239,1.000000,-0.031421,-0.027862,-0.223755,-0.349686
lag1,-0.006062,-0.031421,1.000000,-0.002550,0.002251,0.005108
lag2,0.014415,-0.027862,-0.002550,1.000000,0.023885,0.010641
vol5,-0.220904,-0.223755,0.002251,0.023885,1.000000,0.607844
vol10,-0.344699,-0.349686,0.005108,0.010641,0.607844,1.000000


In [47]:
 data['target'].std()

np.float64(0.0049078985614194924)

In [48]:
y_train_scaled = y_train * 10000
y_test_scaled = y_test * 10000

from lightgbm import LGBMRegressor

model = LGBMRegressor(max_bin=1024)
model.fit(X_train, y_train)

preds = model.predict(X_test)/10000


Tree-based models are not suitable for predicting :

Extremely low variance target 
Features are too smooth / correlated
Small dataset 

Linear regression  is appropriate here.

Linear regression

In [49]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_test)
lr_preds

array([ 7.00099408e-05,  5.39635323e-05,  5.41610893e-05, -5.24259325e-05,
       -5.05941224e-05, -3.10151928e-05,  6.09918156e-06,  1.06331740e-04,
        1.37026100e-04,  1.31004727e-04,  1.45116489e-04,  2.06075722e-04,
        2.50895795e-04,  3.51724880e-04,  4.81288587e-04,  4.84544253e-04,
        2.21942829e-04,  1.14528177e-04,  8.20552804e-05, -6.37896691e-05,
       -1.33202623e-04, -2.39637658e-04, -1.75345219e-04, -1.14396648e-04,
       -8.03053315e-05, -1.12350168e-05,  8.62072807e-05,  7.17993588e-05,
        1.02884945e-05, -1.33950084e-05,  6.62795986e-05,  1.62847278e-04,
        1.46515984e-04,  1.63001449e-04,  3.42448444e-04,  4.04018083e-04,
        3.34688733e-04,  4.97358461e-04,  5.19446124e-04,  5.88981544e-04,
        7.99727107e-04,  9.26455095e-04,  8.02181560e-04,  5.71362412e-04,
        5.48174489e-04,  5.13914260e-04,  5.22335042e-04,  4.51718523e-04,
        3.71317384e-04,  2.27894832e-04,  2.43548335e-04,  2.12921121e-04,
        1.83523861e-04,  

In [50]:
data.head(2)

,index,Date,Close,High,Low,Open,Volume,return,log_return,target,sma5,sma10,lag1,lag2,vol5,vol10
0,30,2015-02-12,1.131196,1.139819,1.130621,1.130825,0,-0.000882,-0.000883,0.008098,1.134945,1.155303,-0.000442,0.001109,0.009672,0.008831
1,31,2015-02-13,1.140394,1.145001,1.138200,1.140485,0,0.008131,0.008098,-0.000342,1.133584,1.153018,-0.000883,-0.000442,0.007937,0.009028


In [51]:
df2=data[['Date','Close']]
df2.head(2)

,Date,Close
0,2015-02-12,1.131196
1,2015-02-13,1.140394


In [52]:
from autots import AutoTS



# Initialize AutoTS
model = AutoTS(
    forecast_length=30,   # predict next 30 days
    frequency='D',        # daily data
    ensemble='simple',    # ensemble option
    max_generations=5,    # for demo; increase for better performance
    validation_method='backwards'
)


Using 5 cpus for n_jobs.


In [63]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data = data.dropna(subset=['Date'])

In [64]:
# Fit the model
model = model.fit(
    data, 
    date_col='Date', 
    value_col='Close', 
    id_col=None
)

# Make predictions
prediction = model.predict()



Data frequency is: B, used frequency is: D
Model Number: 1 with model AverageValueNaive in generation 0 of 5
Model Number: 2 with model AverageValueNaive in generation 0 of 5
Model Number: 3 with model AverageValueNaive in generation 0 of 5
Model Number: 4 with model DatepartRegression in generation 0 of 5
Model Number: 5 with model ETS in generation 0 of 5
Model Number: 6 with model GLM in generation 0 of 5
Model Number: 7 with model GLS in generation 0 of 5
Model Number: 8 with model LastValueNaive in generation 0 of 5
Model Number: 9 with model LastValueNaive in generation 0 of 5
Model Number: 10 with model LastValueNaive in generation 0 of 5
Model Number: 11 with model SeasonalNaive in generation 0 of 5
Model Number: 12 with model SeasonalNaive in generation 0 of 5
Model Number: 13 with model VAR in generation 0 of 5
Template Eval Error: ValueError('Only gave one variable to VAR') in model 13 in generation 0: VAR
Model Number: 14 with model ConstantNaive in generation 0 of 5
Model 

C:\Users\Public\project\venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Public\project\venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\Public\project\venv\Lib\site-packages\statsmodels\tsa\filters\hp_filter.py:100: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  trend = spsolve(I+lamb*K.T.dot(K), x, use_umfpack=use_umfpack)


Model Number: 29 with model Cassandra in generation 0 of 5
Model Number: 30 with model Cassandra in generation 0 of 5
Model Number: 31 with model DMD in generation 0 of 5
Model Number: 32 with model DMD in generation 0 of 5
Model Number: 33 with model SectionalMotif in generation 0 of 5
Model Number: 34 with model ARDL in generation 0 of 5
Model Number: 35 with model FFT in generation 0 of 5
Model Number: 36 with model BasicLinearModel in generation 0 of 5
Template Eval Error: ValueError('maximum supported dimension for an ndarray is currently 64, found 3580') in model 36 in generation 0: BasicLinearModel
Model Number: 37 with model BasicLinearModel in generation 0 of 5
Model Number: 38 with model SeasonalityMotif in generation 0 of 5
Model Number: 39 with model BasicLinearModel in generation 0 of 5
Model Number: 40 with model BallTreeRegressionMotif in generation 0 of 5
Model Number: 41 with model DatepartRegression in generation 0 of 5
Model Number: 42 with model Cassandra in generat

C:\Users\Public\project\venv\Lib\site-packages\scipy\_lib\_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=2.98217e-26): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)


Model Number: 53 with model VAR in generation 0 of 5
Template Eval Error: ValueError('Only gave one variable to VAR') in model 53 in generation 0: VAR
Model Number: 54 with model WindowRegression in generation 0 of 5
Template Eval Error: ValueError("regression_type='User' but no future_regressor passed") in model 54 in generation 0: WindowRegression
Model Number: 55 with model DatepartRegression in generation 0 of 5
Template Eval Error: ValueError('Input X contains NaN.\nElasticNet does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page:

...

C:\Users\Public\project\venv\Lib\site-packages\statsmodels\tsa\filters\hp_filter.py:100: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  trend = spsolve(I+lamb*K.T.dot(K), x, use_umfpack=use_umfpack)


113 - DMD with avg smape 0.68 in 0.09s: 
Model Number: 114 of 112 with model DMD for Validation 3
114 - DMD with avg smape 0.32 in 0.09s: 
Model Number: 115 of 112 with model DMD for Validation 3
115 - DMD with avg smape 0.42 in 0.08s: 
Model Number: 116 of 112 with model BallTreeRegressionMotif for Validation 3
116 - BallTreeRegressionMotif with avg smape 0.77 in 4.22s: 
Model Number: 117 of 112 with model UnivariateMotif for Validation 3
117 - UnivariateMotif with avg smape 0.71 in 0.04s: 
Model Number: 118 of 112 with model MetricMotif for Validation 3
118 - MetricMotif with avg smape 0.54 in 0.03s: 
Model Number: 119 of 112 with model DMD for Validation 3
119 - DMD with avg smape 0.92 in 0.11s: 
Model Number: 120 of 112 with model Cassandra for Validation 3
120 - Cassandra with avg smape 0.48 in 0.21s: 
Model Number: 121 of 112 with model SeasonalNaive for Validation 3
121 - SeasonalNaive with avg smape 0.51 in 0.07s: 
Model Number: 122 of 112 with model DMD for Validation 3
122 - 

C:\Users\Public\project\venv\Lib\site-packages\autots\evaluator\auto_model.py:3073: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  validation_results.model_results = validation_results.model_results.replace(


Model Number: 1176 with model Ensemble in generation 7 of Ensembles
Template Eval Error: Exception("Transformer HistoricValues failed on fit from params fake_date {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 10, 'threshold_method': 'mean'}, '1': {'window': 28}} with error KeyError('2')") in model 1181 in generation 7: Ensemble
Model Number: 1182 with model Ensemble in generation 7 of Ensembles
Template Eval Error: Exception("Transformer HistoricValues failed on fit from params fake_date {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 10, 'threshold_method': 'mean'}, '1': {'window': 28}} with error KeyError('2')") in model 1187 in generation 7: Ensemble
Model Number: 1188 with model Ensemble in generation 7 of Ensembles
Template Eval Error: Exception("Transformer HistoricValues failed on fit from params fake_date {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'stre

C:\Users\Public\project\venv\Lib\site-packages\autots\evaluator\auto_model.py:3073: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  validation_results.model_results = validation_results.model_results.replace(
C:\Users\Public\project\venv\Lib\site-packages\autots\evaluator\auto_model.py:3073: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  validation_results.model_results = validation_results.model_results.replace(


In [29]:
forecast = prediction.forecast
print(forecast.tail())

               Close
2025-01-25  1.031892
2025-01-26  1.031609
2025-01-27  1.031439
2025-01-28  1.031431
2025-01-29  1.036475


In [25]:
# Get the best model details
best_model = model.best_model
print(best_model)


                                  ID     Model  \
84  25582ed3e1edd7a131407b737e09c56c  Ensemble   

                                      ModelParameters  \
84  {"model_name": "BestN", "model_count": 3, "mod...   

                             TransformationParameters  Ensemble  
84  {"fillna": "fake_date", "transformations": {"0...         1  


In [26]:
# Show all tested models and scores
results = model.results()
print(results.head())


                                    ID     Model  \
1066  1e5135b30daac366c6478244e735b2b7  Ensemble   
1073  0e99b717af72182a03cf8b658ca80393  Ensemble   
1080  0e99b717af72182a03cf8b658ca80393  Ensemble   
1087  1e5135b30daac366c6478244e735b2b7  Ensemble   
1065  63f694ac3e2edcca5db0c8d186410b30  Ensemble   

                                        ModelParameters  \
1066  {"model_name": "BestN", "model_count": 3, "mod...   
1073  {"model_name": "BestN", "model_count": 3, "mod...   
1080  {"model_name": "BestN", "model_count": 3, "mod...   
1087  {"model_name": "BestN", "model_count": 3, "mod...   
1065  {"model_name": "BestN", "model_count": 3, "mod...   

                               TransformationParameters TransformationRuntime  \
1066  {"fillna": "fake_date", "transformations": {"0...                0 days   
1073  {"fillna": "fake_date", "transformations": {"0...                0 days   
1080  {"fillna": "fake_date", "transformations": {"0...                0 days   
1087  {"

prophet

In [56]:
df2 = data[['Date', 'Close']].copy()  # create a new DataFrame
df2['Date'] = pd.to_datetime(df2['Date'], errors='coerce')
df2 = df2.rename(columns={'Date': 'ds', 'Close': 'y'})
df2.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [58]:
# Clip extreme outliers if needed
df2['y'] = df2['y'].clip(lower=0)

In [ ]:
from prophet import Prophet
import pandas as pd


p=Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)

p.fit(df2,algorithm='lbfgs', iter=1000)

# Forecast next 30 days
future = p.make_future_dataframe(periods=30)
forecast = p.predict(future)
print(forecast[['ds','yhat','yhat_lower','yhat_upper']].tail())



In [33]:
print(data.isna().sum())



index         0
Date          0
Close         0
High          0
Low           0
Open          0
Volume        0
return        0
log_return    0
target        0
sma5          0
sma10         0
lag1          0
lag2          0
vol5          0
vol10         0
dtype: int64


In [34]:
print((data == float('inf')).sum())

index         0
Date          0
Close         0
High          0
Low           0
Open          0
Volume        0
return        0
log_return    0
target        0
sma5          0
sma10         0
lag1          0
lag2          0
vol5          0
vol10         0
dtype: int64


In [37]:
df2.dtypes

ds     object
y     float64
dtype: object